In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession

val conf = new SparkConf()

val spark = SparkSession
  .builder()
  .appName("Kamaed")
  .config(conf)
  .getOrCreate()

conf = org.apache.spark.SparkConf@3eca2ec2
spark = org.apache.spark.sql.SparkSession@5386cf90


org.apache.spark.sql.SparkSession@5386cf90

In [2]:
spark

org.apache.spark.sql.SparkSession@5386cf90

In [ ]:
spark.sparkContext

In [35]:
import org.apache.spark.sql.functions.{udf, col}

In [3]:
import scala.sys.process._
"hdfs dfs -ls /labs/slaba05/".!!

"Found 2 items
-rw-r--r--   3 hdfs hdfs   24271415 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs  176160768 2022-01-06 18:46 /labs/slaba05/lab05_train.csv
"


In [ ]:
"hdfs dfs -head /labs/slaba05/lab05_train.csv".!!

In [4]:
var df_train = spark.read
        .format("csv")
        .option("header", "true")
        .option("delimiter", ",")
        .option("inferSchema", "true")
        .load("/labs/slaba05/lab05_train.csv")

df_train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [31]:
var df_test = spark.read
        .format("csv")
        .option("header", "true")
        .option("delimiter", ",")
        .option("inferSchema", "true")
        .load("/labs/slaba05/lab05_test.csv")

df_test = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [5]:
df_train = df_train.filter(df_train("ID") !== 369524)

df_train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [6]:
df_train.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [103]:
df_train.summary().show(100, 100, true)

-RECORD 0-------------------------------------------------------
 summary                     | count                            
 _c0                         | 320763                           
 ID                          | 320763                           
 CR_PROD_CNT_IL              | 320763                           
 AMOUNT_RUB_CLO_PRC          | 286213                           
 PRC_ACCEPTS_A_EMAIL_LINK    | 139950                           
 APP_REGISTR_RGN_CODE        | 54850                            
 PRC_ACCEPTS_A_POS           | 139950                           
 PRC_ACCEPTS_A_TK            | 139950                           
 TURNOVER_DYNAMIC_IL_1M      | 320763                           
 CNT_TRAN_AUT_TENDENCY1M     | 69535                            
 SUM_TRAN_AUT_TENDENCY1M     | 69535                            
 AMOUNT_RUB_SUP_PRC          | 286213                           
 PRC_ACCEPTS_A_AMOBILE       | 139950                           
 SUM_TRAN_AUT_TENDENCY3M 

In [106]:
df_train.select(col("PACK")).groupBy(col("PACK")).count().show(100, 100, true)

-RECORD 0-------
 PACK  | 101    
 count | 1645   
-RECORD 1-------
 PACK  | 107    
 count | 25108  
-RECORD 2-------
 PACK  | K01    
 count | 69478  
-RECORD 3-------
 PACK  | M01    
 count | 5      
-RECORD 4-------
 PACK  | 104    
 count | 6073   
-RECORD 5-------
 PACK  | 102    
 count | 105756 
-RECORD 6-------
 PACK  | 103    
 count | 22430  
-RECORD 7-------
 PACK  | 108    
 count | 4      
-RECORD 8-------
 PACK  | 105    
 count | 40777  
-RECORD 9-------
 PACK  | O01    
 count | 45644  
-RECORD 10------
 PACK  | 109    
 count | 83     
-RECORD 11------
 PACK  | 301    
 count | 3760   



In [109]:
val ageGroup = udf((age: Int) => if (age < 288) 0 else if (age < 408) 1 else if (age < 528) 2 else if (age < 648) 3 else 4);

ageGroup = UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))

In [111]:
df_train.select(ageGroup(col("AGE")).alias("AG")).groupBy(col("AG")).count().show(10, 100, true)

-RECORD 0-------
 AG    | 1      
 count | 114443 
-RECORD 1-------
 AG    | 3      
 count | 57486  
-RECORD 2-------
 AG    | 4      
 count | 36889  
-RECORD 3-------
 AG    | 2      
 count | 92820  
-RECORD 4-------
 AG    | 0      
 count | 19125  



# Шаг 1: Кодируем возраст

In [114]:
val df_train_1 = df_train.withColumn("AGE_G", ageGroup(col("AGE")))

df_train_1 = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [115]:
val df_test_1 = df_test.withColumn("AGE_G", ageGroup(col("AGE")))

df_test_1 = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [ ]:
df_train.filter(df_train("TARGET") === 0).summary().show(100, 100, true)

In [ ]:
df_train.filter(df_train("TARGET") === 1).summary().show(100, 100, true)

In [ ]:
df_train.select("TURNOVER_CC",
"LDEAL_GRACE_DAYS_PCT_MED",
"REST_DYNAMIC_CC_1M",
"LDEAL_AMT_MONTH",
"TURNOVER_DYNAMIC_CC_1M",
"REST_DYNAMIC_CC_3M",
"AVG_PCT_DEBT_TO_DEAL_AMT",
"TURNOVER_DYNAMIC_CC_3M",
"REST_DYNAMIC_IL_1M",
"CR_PROD_CNT_CCFP",
"REST_AVG_PAYM",
"REST_AVG_CUR",
"TURNOVER_PAYM",
"LDEAL_DELINQ_PER_MAXYWZ",
"REST_DYNAMIC_IL_3M",
"TURNOVER_DYNAMIC_IL_1M",
"REST_DYNAMIC_FDEP_1M",
"DEAL_GRACE_DAYS_ACC_MAX",
"DEAL_GRACE_DAYS_ACC_AVG",
"TURNOVER_DYNAMIC_IL_3M",
"REST_DYNAMIC_FDEP_3M",
"CR_PROD_CNT_VCU",
"DEAL_GRACE_DAYS_ACC_S1X1",
"CR_PROD_CNT_CC",
"AMOUNT_RUB_NAS_PRC",
"PACK").summary().show(100, 100, true)

In [ ]:
df_train.select("ID").distinct().count()

In [ ]:
df_train.select("ID").count()

ID правда уникальный

# Смотрим данные

Пропорции классов

In [ ]:
df_train.filter(df_train("TARGET") === 0).count()

In [ ]:
df_train.filter(df_train("TARGET") === 1).count()

Кол-во открытых продуктов за отчетный период (по категориям)

CR_PROD_CNT_CC, CR_PROD_CNT_CCFP, CR_PROD_CNT_IL, CR_PROD_CNT_PIL, CR_PROD_CNT_TOVR, CR_PROD_CNT_VCU

In [ ]:
df_train.select("CR_PROD_CNT_CC", "CR_PROD_CNT_CCFP", "CR_PROD_CNT_IL", "CR_PROD_CNT_PIL", "CR_PROD_CNT_TOVR", "CR_PROD_CNT_VCU").summary().show()

In [ ]:
df_train.filter(df_train("CR_PROD_CNT_CC").isNull).take(10)

In [ ]:
df_train = df_train.filter(df_train("ID") !== 369524)

Удалили бесполезную строчку - у нее target не было

### Доля суммы транзакций по МСС ко сумме всех транзакций за период (в руб.)

AMOUNT_RUB_ATM_PRC, AMOUNT_RUB_CLO_PRC, AMOUNT_RUB_NAS_PRC, AMOUNT_RUB_SUP_PRC

In [ ]:
df_train.select("AMOUNT_RUB_ATM_PRC", "AMOUNT_RUB_CLO_PRC", "AMOUNT_RUB_NAS_PRC", "AMOUNT_RUB_SUP_PRC").summary().show()

Чета есть, но почему только 286213?

In [ ]:
df_train.filter(df_train("AMOUNT_RUB_ATM_PRC").isNull).select("ID", "AMOUNT_RUB_ATM_PRC", "AMOUNT_RUB_CLO_PRC", "AMOUNT_RUB_NAS_PRC", "AMOUNT_RUB_SUP_PRC", "TARGET").show(3, 35, true)

In [ ]:
df_train.filter(df_train("AMOUNT_RUB_ATM_PRC").isNotNull).select("ID", "AMOUNT_RUB_ATM_PRC", "AMOUNT_RUB_CLO_PRC", "AMOUNT_RUB_NAS_PRC", "AMOUNT_RUB_SUP_PRC", "TARGET").show(3, 35, true)

### % отклика в каналах / продуктовых группах

PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_MTP, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, PRC_ACCEPTS_MTP, PRC_ACCEPTS_TK

In [ ]:
df_train.select("PRC_ACCEPTS_A_AMOBILE", "PRC_ACCEPTS_A_ATM", "PRC_ACCEPTS_A_EMAIL_LINK", "PRC_ACCEPTS_A_MTP", "PRC_ACCEPTS_A_POS", "PRC_ACCEPTS_A_TK", "PRC_ACCEPTS_MTP", "PRC_ACCEPTS_TK").summary().show()

Одни нули, можно дропать

### Код региона регистрации

APP_REGISTR_RGN_CODE

In [ ]:
df_train.select("APP_REGISTR_RGN_CODE").summary().show()

По смыслу можно дропнуть

# Анализ колонок

Провел в питоне)

Решил оставить только эти: 
"TURNOVER_CC",
"LDEAL_GRACE_DAYS_PCT_MED",
"REST_DYNAMIC_CC_1M",
"TURNOVER_DYNAMIC_CC_1M",
"REST_DYNAMIC_CC_3M",
"TURNOVER_DYNAMIC_CC_3M",
"REST_DYNAMIC_IL_1M",
"CR_PROD_CNT_CCFP",
"REST_AVG_PAYM",
"REST_AVG_CUR",
"TURNOVER_PAYM",
"REST_DYNAMIC_IL_3M",
"TURNOVER_DYNAMIC_IL_1M",
"REST_DYNAMIC_FDEP_1M",
"TURNOVER_DYNAMIC_IL_3M",
"REST_DYNAMIC_FDEP_3M",
"CR_PROD_CNT_VCU",
"CR_PROD_CNT_CC"

In [112]:
import org.apache.spark.ml.feature.OneHotEncoder

In [126]:
val encoder = new OneHotEncoder()
    .setInputCol("AGE_G")
    .setOutputCol("AGE_VEC")

encoder = oneHot_98acb952c23c


oneHot_98acb952c23c

In [124]:
val encoder_model = encoder.transform(df_train_1)

encoder_model = [_c0: int, ID: int ... 117 more fields]


[_c0: int, ID: int ... 117 more fields]

In [125]:
encoder_model.select(col("AGE_G"), col("AGE_VEC")).take(10)

Array([1,(5,[1],[1.0])], [1,(5,[1],[1.0])], [3,(5,[3],[1.0])], [4,(5,[4],[1.0])], [2,(5,[2],[1.0])], [2,(5,[2],[1.0])], [2,(5,[2],[1.0])], [3,(5,[3],[1.0])], [2,(5,[2],[1.0])], [2,(5,[2],[1.0])])

In [7]:
import org.apache.spark.ml.feature.VectorAssembler

In [127]:
val assembler = new VectorAssembler()
  .setInputCols(Array(
    "CR_PROD_CNT_IL",
    "TURNOVER_DYNAMIC_IL_1M",
    "REST_DYNAMIC_FDEP_1M",
    "REST_DYNAMIC_SAVE_3M",
    "CR_PROD_CNT_VCU",
    "REST_AVG_CUR",
    "CR_PROD_CNT_TOVR",
    "CR_PROD_CNT_PIL",
    "TURNOVER_CC",
    "TURNOVER_PAYM",
    "CR_PROD_CNT_CC",
    "REST_DYNAMIC_FDEP_3M",
    "REST_DYNAMIC_IL_1M",
    "CR_PROD_CNT_CCFP",
    "REST_DYNAMIC_CUR_1M",
    "REST_AVG_PAYM",
    "LDEAL_GRACE_DAYS_PCT_MED",
    "REST_DYNAMIC_CUR_3M",
    "TURNOVER_DYNAMIC_CUR_1M",
    "REST_DYNAMIC_PAYM_3M",
    "REST_DYNAMIC_IL_3M",
    "TURNOVER_DYNAMIC_IL_3M",
    "REST_DYNAMIC_PAYM_1M",
    "TURNOVER_DYNAMIC_CUR_3M",
    "CLNT_SETUP_TENOR",
    "TURNOVER_DYNAMIC_PAYM_3M",
    "TURNOVER_DYNAMIC_PAYM_1M",
    "REST_DYNAMIC_CC_1M",
    "TURNOVER_DYNAMIC_CC_1M",
    "REST_DYNAMIC_CC_3M",
    "TURNOVER_DYNAMIC_CC_3M",
    "AGE_VEC"))
  .setOutputCol("features")

assembler = vecAssembler_4baf16921753


vecAssembler_4baf16921753

In [9]:
import org.apache.spark.ml.feature.StandardScaler

In [10]:
val scaler = new StandardScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")
  .setWithStd(true)
  .setWithMean(true)

scaler = stdScal_fa47eca20d7d


stdScal_fa47eca20d7d

In [13]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}

In [90]:
val gbt = new GBTClassifier()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setMaxIter(10)
  .setFeatureSubsetStrategy("auto")

gbt = gbtc_0331b4d8397d


gbtc_0331b4d8397d

In [128]:
val pipeline = new Pipeline()
  .setStages(Array(encoder, assembler, gbt))

pipeline = pipeline_16b907025425


pipeline_16b907025425

In [129]:
val model = pipeline.fit(df_train_1)

model = pipeline_16b907025425


pipeline_16b907025425

In [130]:
val predictions = model.transform(df_test_1)

predictions = [_c0: int, ID: int ... 120 more fields]


[_c0: int, ID: int ... 120 more fields]

In [33]:
predictions.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [58]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.mllib.linalg.Vectors

In [71]:
val getProb = udf((probArr: Vector) => probArr(1))

getProb = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [131]:
val res = predictions.select(col("ID").as("id"), getProb(col("probability")).as("target"))

res = [id: int, target: double]


[id: int, target: double]

In [133]:
res.repartition(1).write
    .option("header", true)
    .option("delimiter","\t")
    .csv("lab05.csv")

In [85]:
"hdfs dfs -ls".!!

"Found 9 items
drwx------   - dmitriy.kamaev dmitriy.kamaev          0 2022-11-02 15:35 .Trash
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-11-02 12:15 .sparkStaging
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-11-02 15:36 lab05.csv
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-10-27 13:08 sdf_5.parquet
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-10-27 13:10 sdf_6.parquet
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-10-27 13:11 sdf_7.parquet
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-10-26 20:49 ss_01.parquet
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-10-27 14:13 streaming
drwxr-xr-x   - dmitriy.kamaev dmitriy.kamaev          0 2022-10-26 20:48 tmp
"


In [132]:
"hdfs dfs -rm -r lab05.csv".!!

22/11/02 20:54:03 INFO fs.TrashPolicyDefault: Moved: 'hdfs://spark-master-1.newprolab.com:8020/user/dmitriy.kamaev/lab05.csv' to trash at: hdfs://spark-master-1.newprolab.com:8020/user/dmitriy.kamaev/.Trash/Current/user/dmitriy.kamaev/lab05.csv1667411643859


res205: String = ""


In [134]:
"hdfs dfs -ls lab05.csv".!!

"Found 2 items
-rw-r--r--   3 dmitriy.kamaev dmitriy.kamaev          0 2022-11-02 20:54 lab05.csv/_SUCCESS
-rw-r--r--   3 dmitriy.kamaev dmitriy.kamaev    1186492 2022-11-02 20:54 lab05.csv/part-00000-02b6b81c-bbc5-4858-80d5-3edf3da04393-c000.csv
"


In [136]:
"rm lab05.csv".!!

lastException: Throwable = null
res212: String = ""


In [137]:
"hdfs dfs -copyToLocal lab05.csv/part-00000-02b6b81c-bbc5-4858-80d5-3edf3da04393-c000.csv lab05.csv".!!

res215: String = ""


---

In [138]:
spark.stop()